## tensorflow を使った単層(中間層1層）ニューラルネットワーク　の　ensorBoard 表示

## 以下は，単層(中間層1層）ニューラルネットワーク　（JIEP_tensorflow_nn_tanso-MNIST.ipynb）と同じ内容です．

## まずは，実行してみてください．

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160614)
tf.set_random_seed(20160614)

mnist = input_data.read_data_sets("/tmp/data", one_hot=True)


class SingleLayerNetwork:
    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()
            
    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
            
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]),
                             name = 'weights')
            b1 = tf.Variable(tf.zeros([num_units]), name ='biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1, name = 'hidden1')
            
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]), name = 'weights')
            b0 = tf.Variable(tf.zeros([10]), name = 'biases')
            p  = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name = 'softmax')
            
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name = 'labels')
            loss = -tf.reduce_sum(t * tf.log(p), name = 'loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)
        
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, 
                                              tf.float32), name = 'accuracy')

#以下，tf.scalar_summary -> tf.summary.scalar の修正を行った．
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
#以下，tf.histogram_summary -> tf.summary.histogram の修正を行った．
        tf.summary.histogram("weights_hidden", w1)
        tf.summary.histogram("biases_hidden", b1)
        tf.summary.histogram("weights_output", w0)
        tf.summary.histogram("biases_output", b0)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy

    def prepare_session(self):
        sess = tf.InteractiveSession()
# sess.run(tf.initialize_all_variables()) # 2017-03-02 より以下に変更
        sess.run(tf.global_variables_initializer())
        
#以下，tf.merge_all_summaries -> tf.summary.merge_all の修正を行った．
        summary = tf.summary.merge_all()
# 以下，tf.train.SummaryWriter -> tf.summary.FileWriter の修正を行った．
        writer = tf.summary.FileWriter("/tmp/mnist_sl_logs", sess.graph)      
    
    
        self.sess = sess
        self.summary = summary
        self.writer = writer
    
#!rm -rf /tmp/mnist_sl_logs


nn = SingleLayerNetwork(512)

i=0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x: batch_xs, nn.t: batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run(
            [nn.summary, nn.loss, nn.accuracy],
            feed_dict={nn.x:mnist.test.images, nn.t: mnist.test.labels})
        print ('Step:  %d, Loss: %f, Accuracy: %f'
              % (i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)
        
        

        

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Step:  100, Loss: 3089.492676, Accuracy: 0.914200
Step:  200, Loss: 2572.291504, Accuracy: 0.921100
Step:  300, Loss: 2256.864502, Accuracy: 0.931100
Step:  400, Loss: 1831.467896, Accuracy: 0.945700
Step:  500, Loss: 1756.163940, Accuracy: 0.946400
Step:  600, Loss: 1725.195679, Accuracy: 0.948400
Step:  700, Loss: 1554.253174, Accuracy: 0.950700
Step:  800, Loss: 1477.438721, Accuracy: 0.955000
Step:  900, Loss: 1430.414551, Accuracy: 0.955400
Step:  1000, Loss: 1482.965332, Accuracy: 0.952800
Step:  1100, Loss: 1375.742432, Accuracy: 0.957700
Step:  1200, Loss: 1243.835693, Accuracy: 0.962400
Step:  1300, Loss: 1258.420166, Accuracy: 0.960700
Step:  1400, Loss: 1192.930786, Accuracy: 0.962300
Step:  1500, Loss: 1262.994019, Accuracy: 0.961700
Step:  1600, Loss: 1200.536499, Accuracy: 0.963700
Ste

## /tmp/data の下に mnist_sl_logs というフォルダができています．

## そのフォルダの中に　events.out.tfevents.xxxxxx　というファイルができています．

## 最新のファイルだけのこして，あとは削除してください．

## anaconda から ターミナルを立ち上げてください．

## ターミナルのプロンプトで 

##              tensorboard –logdir=/tmp/mnist_sl_logs

## と入力してください．すると，ターミナルに

##              http://XXXXXX:6006

## と表示されます．このURLをブラウザに入力してください．